In [1]:
!pip install symspellpy
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=187467 sha256=3075c4bc476b74201df974f424bbcaf4a6d09d02f4fbf207125bb2f326c4d25f
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 15.6 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.3 MB/s eta 0:00:00


In [3]:
import os
import pkg_resources
from symspellpy import SymSpell, Verbosity
import easyocr

# Initialize SymSpell

In [4]:
sym_spell = SymSpell(max_dictionary_edit_distance=2)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

# Initialize EasyOCR Reader

In [10]:
def recognize_text_with_correction(img_path):
    reader = easyocr.Reader(['en'], gpu=False, verbose=False)
    result = reader.readtext(img_path)

    valid_document_found = False

    for i in range(len(result)):
        for j in range(len(result[i])):
            current_text = str(result[i][j]).lower()
            suggestions = sym_spell.lookup_compound(current_text, 2)
            corrected_text = suggestions[0].term if suggestions else current_text

            if "income tax department" in corrected_text or "permanent account number" in corrected_text or "govt of india" in corrected_text:
                print(f'Image: {img_path}, Text: Pan Card')
                valid_document_found = True
                break
            elif "government of india" in corrected_text:
                print(f'Image: {img_path}, Text: Aadhar Card')
                valid_document_found = True
                break
            elif "driving licence" in corrected_text:
                print(f'Image: {img_path}, Text: Driving Licence')
                valid_document_found = True
                break
            elif "election" in corrected_text:
                print(f'Image: {img_path}, Text: Voter ID')
                valid_document_found = True
                break
            elif "republic" in corrected_text or "spouse" in corrected_text or "old passport" in corrected_text:
                print(f'Image: {img_path}, Text: Passport')
                valid_document_found = True
                break

        if valid_document_found:
            break

    if not valid_document_found:
        print(f'Image: {img_path}, Text: Not a valid document')

In [6]:
import os
import fitz
from PIL import Image

colab_directory = '/content/'

def convert_pdf_to_images(pdf_path, output_directory):
    images = []
    pdf_document = fitz.open(pdf_path)

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        image = page.get_pixmap()
        images.append(Image.frombytes("RGB", [image.width, image.height], image.samples))

    pdf_document.close()

    converted_paths = [] # Save the converted images to the specified output directory
    for i, img in enumerate(images):
        page_filename = f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{i}.png"
        output_path = os.path.join(output_directory, page_filename)
        img.save(output_path)
        converted_paths.append(output_path)
        print(f"Converted and saved: {output_path}")

    return converted_paths

converted_directory = os.path.join(colab_directory, 'converted_images') # Create a subdirectory for converted images
os.makedirs(converted_directory, exist_ok=True)

image_paths = []

for filename in os.listdir(colab_directory):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
        image_path = os.path.join(colab_directory, filename)
        image_paths.append(image_path)
    elif filename.lower().endswith('.pdf'):
        pdf_path = os.path.join(colab_directory, filename)
        converted_paths = convert_pdf_to_images(pdf_path, converted_directory)
        image_paths.extend(converted_paths)

print(image_paths)

Converted and saved: /content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_0.png
Converted and saved: /content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_1.png
['/content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_0.png', '/content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_1.png', '/content/908d00ff-87bc-4fc8-95dd-9d621265ddca.jpeg', '/content/Aadhar.png', '/content/61153ad9-f342-4164-947b-b03982a5fcab.jpeg', '/content/c638138d-dc91-43e2-83a3-a49776f025cb.jpeg']


In [11]:
for img_path in image_paths:
    recognize_text_with_correction(img_path)

Image: /content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_0.png, Text: Passport
Image: /content/converted_images/d98e9580-fd10-4e0b-bd4c-0855b40b253b_page_1.png, Text: Passport
Image: /content/908d00ff-87bc-4fc8-95dd-9d621265ddca.jpeg, Text: Driving Licence
Image: /content/Aadhar.png, Text: Aadhar Card
Image: /content/61153ad9-f342-4164-947b-b03982a5fcab.jpeg, Text: Pan Card
Image: /content/c638138d-dc91-43e2-83a3-a49776f025cb.jpeg, Text: Voter ID
